# Anime LR Problem

Import the data

In [16]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName('Anime').getOrCreate()
sc = spark.sparkContext

import numpy as np
import pandas as pd

In [17]:
# read data
df = spark.read.option("header","true").csv("anime.csv",inferSchema = True)

Preprocess the data

In [18]:
# drop NA value
df = df[df['Score'] != 'Unknown']
df = df[df['Episodes'] != 'Unknown']

In [19]:
# convert string to float & integer
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType
df = df.withColumn('Score',col('Score').cast('float'))
df = df.withColumn('Episodes',col('Episodes').cast('int'))
df = df.withColumn('Popularity',col('Popularity').cast('int'))
df = df.dropna()

In [20]:
# create a new column 'Views'
df = df.withColumn('Views',df['Score-10']+df['Score-9']+df['Score-8']+df['Score-7']+df['Score-6']
                  +df['Score-5']+df['Score-4']+df['Score-3']+df['Score-2']+df['Score-1'])

In [21]:
# select column
df = df.select(['Score','Type','Episodes','Source','Rating','Popularity','Members','Favorites','Watching',
                'On-Hold','Dropped','Plan to Watch','Views'])
df.show(5)

+-----+-----+--------+--------+--------------------+----------+---------+---------+--------+-------+-------+-------------+--------+
|Score| Type|Episodes|  Source|              Rating|Popularity|  Members|Favorites|Watching|On-Hold|Dropped|Plan to Watch|   Views|
+-----+-----+--------+--------+--------------------+----------+---------+---------+--------+-------+-------+-------------+--------+
| 8.78|   TV|      26|Original|R - 17+ (violence...|        39|1251960.0|    61971|  105808|  71513|  26678|       329800|641705.0|
| 8.39|Movie|       1|Original|R - 17+ (violence...|       518| 273145.0|     1174|    4143|   1935|    770|        57964|160349.0|
| 8.24|   TV|      26|   Manga|PG-13 - Teens 13 ...|       201| 558913.0|    12944|   29113|  25465|  13925|       146918|286146.0|
| 7.27|   TV|      26|Original|PG-13 - Teens 13 ...|      1467|  94683.0|      587|    4300|   5121|   5378|        33719| 39094.0|
| 6.98|   TV|      52|   Manga|       PG - Children|      4369|  13224.0|   

In [22]:
# x and y variables
df.printSchema()

root
 |-- Score: float (nullable = true)
 |-- Type: string (nullable = true)
 |-- Episodes: integer (nullable = true)
 |-- Source: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Popularity: integer (nullable = true)
 |-- Members: double (nullable = true)
 |-- Favorites: integer (nullable = true)
 |-- Watching: integer (nullable = true)
 |-- On-Hold: integer (nullable = true)
 |-- Dropped: integer (nullable = true)
 |-- Plan to Watch: integer (nullable = true)
 |-- Views: double (nullable = true)



In [23]:
# summary
df.summary().toPandas()

,summary,Score,Type,Episodes,Source,Rating,Popularity,Members,Favorites,Watching,On-Hold,Dropped,Plan to Watch,Views
0,count,12312,12312,12312,12312,12312,12312,12312,12312,12312,12312,12312,12312,12008
1,mean,6.510845519287622,None,11.328866146848602,None,None,6437.678525016245,48716.99918778428,638.0909681611436,3044.6125730994154,1328.1386452241716,1633.9566276803118,11225.28248862898,25837.245419720188
2,stddev,0.887440196112299,None,42.804102187507304,None,None,3851.467577178708,146707.64126986082,4703.773391208465,14389.574512895428,4701.004098135914,5350.285755354186,27362.514111349985,88884.16350757869
3,min,1.85,Movie,1,4-koma manga,G - All Ages,1,172.0,0,0,0,0,12,101.0
4,25%,5.93,None,1,None,None,3129,1488.0,2,53,33,61,514,542.0
5,50%,6.52,None,2,None,None,6320,5326.0,10,216,133,144,1736,2145.0
6,75%,7.14,None,12,None,None,9585,27411.0,72,1014,567,548,7597,12394.0
7,max,9.19,TV,1818,Web manga,Unknown,15374,2589552.0,183914,566239,130961,174710,425531,1826691.0


Linear Regression

In [24]:
# convert string variables
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml import Pipeline
convert = [StringIndexer(inputCol = column, outputCol = column+"_index")
            .fit(df) for column in ['Type','Source','Rating']]
pipeline = Pipeline(stages = convert)
df = pipeline.fit(df).transform(df)
df = df.drop('Type','Source','Rating')
df.show(10)

+-----+--------+----------+---------+---------+--------+-------+-------+-------------+--------+----------+------------+------------+
|Score|Episodes|Popularity|  Members|Favorites|Watching|On-Hold|Dropped|Plan to Watch|   Views|Type_index|Source_index|Rating_index|
+-----+--------+----------+---------+---------+--------+-------+-------+-------------+--------+----------+------------+------------+
| 8.78|      26|        39|1251960.0|    61971|  105808|  71513|  26678|       329800|641705.0|       0.0|         1.0|         3.0|
| 8.39|       1|       518| 273145.0|     1174|    4143|   1935|    770|        57964|160349.0|       2.0|         1.0|         3.0|
| 8.24|      26|       201| 558913.0|    12944|   29113|  25465|  13925|       146918|286146.0|       0.0|         0.0|         0.0|
| 7.27|      26|      1467|  94683.0|      587|    4300|   5121|   5378|        33719| 39094.0|       0.0|         1.0|         0.0|
| 6.98|      52|      4369|  13224.0|       18|     642|    766|   11

In [25]:
# create vector
feature = VectorAssembler(inputCols = df.columns[1:],outputCol = "Features")
feature_vector = feature.transform(df)

In [26]:
# split data to train and test subset
(traindata,testdata) = feature_vector.randomSplit([0.7, 0.3],seed = 24)

In [27]:
# model
from pyspark.ml.regression import LinearRegression
score_lr = LinearRegression(featuresCol = 'Features',labelCol = 'Score')
train_model = score_lr.fit(traindata)
results = train_model.evaluate(traindata)

22/12/03 16:29:05 WARN Instrumentation: [201a2ba6] regParam is zero, which might cause numerical instability and overfitting.
22/12/03 16:29:05 ERROR Executor: Exception in task 1.0 in stage 25.0 (TID 37)
org.apache.spark.SparkException: Failed to execute user defined function(VectorAssembler$$Lambda$3359/0x0000000801420040: (struct<Episodes_double_VectorAssembler_4315e48d728b:double,Popularity_double_VectorAssembler_4315e48d728b:double,Members:double,Favorites_double_VectorAssembler_4315e48d728b:double,Watching_double_VectorAssembler_4315e48d728b:double,On-Hold_double_VectorAssembler_4315e48d728b:double,Dropped_double_VectorAssembler_4315e48d728b:double,Plan to Watch_double_VectorAssembler_4315e48d728b:double,Views:double,Type_index:double,Source_index:double,Rating_index:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.sort_addToSorter_0$(Unknown Source)
	a

Py4JJavaError: An error occurred while calling o559.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 25.0 failed 1 times, most recent failure: Lost task 1.0 in stage 25.0 (TID 37) (10.232.185.218 executor driver): org.apache.spark.SparkException: Failed to execute user defined function(VectorAssembler$$Lambda$3359/0x0000000801420040: (struct<Episodes_double_VectorAssembler_4315e48d728b:double,Popularity_double_VectorAssembler_4315e48d728b:double,Members:double,Favorites_double_VectorAssembler_4315e48d728b:double,Watching_double_VectorAssembler_4315e48d728b:double,On-Hold_double_VectorAssembler_4315e48d728b:double,Dropped_double_VectorAssembler_4315e48d728b:double,Plan to Watch_double_VectorAssembler_4315e48d728b:double,Views:double,Type_index:double,Source_index:double,Rating_index:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCo